In [67]:
# Import Packages
import numpy as np
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os
import pandas as pd

# ------------------------------
# Helper function to load images
# ------------------------------
def load_image(path):
    img = sitk.ReadImage(path)
    return sitk.GetArrayFromImage(img), img


# ==========================================================
# GUI FUNCTIONS
# ==========================================================
def select_dose_file():
    path = filedialog.askopenfilename(
        title="Select the calibrated Dose image (NIfTI / MHD)",
        filetypes=[("NIfTI", "*.nii *.nii.gz"), ("MetaImage", "*.mhd"), ("All files", "*.*")]
    )
    if path:
        entry_dose.delete(0, tk.END)
        entry_dose.insert(0, path)

def select_unc_file():
    path = filedialog.askopenfilename(
        title="Select Relative Statistical Uncertainty image from MC-GATE (NIfTI / MHD)",
        filetypes=[("NIfTI", "*.nii *.nii.gz"), ("MetaImage", "*.mhd"), ("All files", "*.*")]
    )
    if path:
        entry_unc.delete(0, tk.END)
        entry_unc.insert(0, path)

def select_voi_folder():
    path = filedialog.askdirectory(title="Select VOI folder (contains VOI .nii/.nii.gz files)")
    if path:
        entry_voi.delete(0, tk.END)
        entry_voi.insert(0, path)

def run_calculation():
    dose_path = entry_dose.get().strip()
    unc_path = entry_unc.get().strip()
    voi_folder = entry_voi.get().strip()

    if not dose_path or not unc_path or not voi_folder:
        messagebox.showerror("Error", "Please select Dose file, Relative Uncertainty file and VOI folder.")
        return

    try:
        # Load images
        dose_array, dose_img = load_image(dose_path)
        unc_array, _ = load_image(unc_path)

        # Check shape match
        if dose_array.shape != unc_array.shape:
            messagebox.showerror("Error", "Dose and Relative Uncertainty images do not match!")
            return

        # Basic sanity: uncertainties should be non-negative
        if np.any(unc_array < 0):
            messagebox.showwarning(
                "Warning",
                "Relative uncertainty image contains negative values. "
                "This is unusual; results may be invalid."
            )

        results = []

        # Loop over VOIs
        for filename in sorted(os.listdir(voi_folder)):
            if not (filename.endswith(".nii") or filename.endswith(".nii.gz") or filename.endswith(".mhd")):
                continue

            voi_path = os.path.join(voi_folder, filename)
            voi_array, _ = load_image(voi_path)

            # Shape check
            if voi_array.shape != dose_array.shape:
                print(f"Skipping {filename}: shape mismatch")
                continue

            mask = voi_array > 0
            n_voxels = int(np.count_nonzero(mask))

            if n_voxels == 0:
                continue

            # Extract voxel values inside VOI
            Di = dose_array[mask].astype(np.float64)
            ui = unc_array[mask].astype(np.float64)

            # Mean absorbed dose in VOI (unweighted voxel mean)
            mean_dose = float(np.mean(Di))

            # Absolute statistical uncertainty per voxel: sigma_i = u_i * D_i
            sigma_i = ui * Di

            # ---- KEY FIX ----
            # Absolute statistical uncertainty (standard deviation) of the VOI mean dose:
            # sigma_mean = sqrt(sum(sigma_i^2)) / N  (assuming independent voxel estimators)
            sigma_mean = float(np.sqrt(np.sum(sigma_i ** 2)) / n_voxels)

            # Relative statistical uncertainty (%) of the VOI mean
            rel_unc_percent = float((sigma_mean / mean_dose) * 100.0) if mean_dose != 0 else 0.0

            # OPTIONAL: spatial SD (dose heterogeneity) within VOI
            # This is NOT MC uncertainty; it's dose non-uniformity across voxels
            spatial_sd = float(np.std(Di, ddof=1)) if n_voxels > 1 else 0.0

            results.append({
                "VOI": filename,
                "Voxels": n_voxels,
                "Mean Absorbed Dose (Gy)": mean_dose,
                "MC Abs. SU of Mean (Gy)": sigma_mean,
                "MC Rel. SU of Mean (%)": rel_unc_percent,
                "Spatial SD in VOI (Gy)": spatial_sd
            })

        # Convert results to DataFrame
        df = pd.DataFrame(results)

        if df.empty:
            messagebox.showinfo("Results", "No VOIs found or all VOIs were skipped (shape mismatch).")
            return

        # Format for display (keep numeric for CSV, but show nice in table)
        df_display = df.copy()
        df_display["Mean Absorbed Dose (Gy)"] = df_display["Mean Absorbed Dose (Gy)"].map(lambda x: f"{x:.6g}")
        df_display["MC Abs. SU of Mean (Gy)"] = df_display["MC Abs. SU of Mean (Gy)"].map(lambda x: f"{x:.6g}")
        df_display["MC Rel. SU of Mean (%)"] = df_display["MC Rel. SU of Mean (%)"].map(lambda x: f"{x:.6g}")
        df_display["Spatial SD in VOI (Gy)"] = df_display["Spatial SD in VOI (Gy)"].map(lambda x: f"{x:.6g}")

        # Display results in table
        display_table(df_display)

        # Save to CSV (numeric values, not formatted strings)
        csv_path = os.path.join(voi_folder, "VOI_MC_statistical_uncertainty_results.csv")
        df.to_csv(csv_path, index=False)

        messagebox.showinfo("Done", f"Results saved to:\n{csv_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred:\n{e}")


def display_table(df):
    # Clear previous table if exists
    for widget in frame_table.winfo_children():
        widget.destroy()

    table = ttk.Treeview(frame_table)
    table["columns"] = list(df.columns)
    table["show"] = "headings"

    for col in df.columns:
        table.heading(col, text=col)
        table.column(col, anchor="center", width=220)

    for _, row in df.iterrows():
        table.insert("", "end", values=list(row))

    table.pack(fill="both", expand=True)


# ==========================================================
# --- MAIN WINDOW SETUP ---
# ==========================================================
root = tk.Tk()
root.title("MC-GATE VOI MC Statistical Uncertainty Calculator")

# Window & style
root.geometry("1300x850")
root.resizable(True, True)
root.configure(bg="#f2f4f7")

style = ttk.Style()
style.configure("Treeview", font=("Arial", 12))
style.configure("Treeview.Heading", font=("Arial", 13, "bold"))

# --- Dose selection ---
tk.Label(root, text="Select the calibrated Dose image (Gy) (NIfTI / MHD):", bg="#f2f4f7").pack(anchor="w", padx=10, pady=(10, 0))
frame_dose = tk.Frame(root, bg="#f2f4f7")
frame_dose.pack(fill="x", padx=10)

entry_dose = tk.Entry(frame_dose, width=70)
entry_dose.pack(side="left", fill="x", expand=True)

tk.Button(frame_dose, text="Browse", command=select_dose_file, bg="#dcdcdc").pack(side="right", padx=5)

# --- Uncertainty selection ---
tk.Label(root, text="Select Relative Statistical Uncertainty image (fraction or %) (NIfTI / MHD):", bg="#f2f4f7").pack(anchor="w", padx=10, pady=(10, 0))
frame_unc = tk.Frame(root, bg="#f2f4f7")
frame_unc.pack(fill="x", padx=10)

entry_unc = tk.Entry(frame_unc, width=70)
entry_unc.pack(side="left", fill="x", expand=True)

tk.Button(frame_unc, text="Browse", command=select_unc_file, bg="#dcdcdc").pack(side="right", padx=5)

# --- VOI folder selection ---
tk.Label(root, text="Select VOI folder (contains VOI masks .nii/.nii.gz/.mhd):", bg="#f2f4f7").pack(anchor="w", padx=10, pady=(10, 0))
frame_voi = tk.Frame(root, bg="#f2f4f7")
frame_voi.pack(fill="x", padx=10)

entry_voi = tk.Entry(frame_voi, width=70)
entry_voi.pack(side="left", fill="x", expand=True)

tk.Button(frame_voi, text="Browse", command=select_voi_folder, bg="#dcdcdc").pack(side="right", padx=5)

# --- Compute button ---
tk.Button(
    root,
    text="Compute VOI Mean Dose and MC Statistical Uncertainty",
    command=run_calculation,
    bg="#2e8b57",
    fg="white",
    height=2,
    font=("Arial", 11, "bold")
).pack(pady=15)

# --- Results table frame ---
frame_table = tk.Frame(root, bg="#f2f4f7")
frame_table.pack(fill="both", expand=True, padx=10, pady=10)

root.mainloop()